In [4]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.callbacks import TensorBoard
import tensorflow as tf

In [5]:
# Implement from https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
def as_keras_metric(method):
    import functools
    from keras import backend as K
    import tensorflow as tf
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper

auc_roc = as_keras_metric(tf.metrics.auc)
recall = as_keras_metric(tf.metrics.recall)
precision = as_keras_metric(tf.metrics.precision)


# dimensions of our images.
img_width, img_height = 250, 250

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 60

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=[auc_roc,recall,precision,'accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

tensorboard = TensorBoard(log_dir='./graph', histogram_freq=0,  
          write_graph=True, write_images=True)    

 


Found 354 images belonging to 5 classes.
Found 33 images belonging to 5 classes.


In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    callbacks=[tensorboard],
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)
model.save('classify.h5')
model.save_weights('classify_weight.h5')

Epoch 1/50
33/33 [==============================] - 63s 2s/step - loss: -12.5988 - auc: 0.4895 - recall: 0.9697 - precision: 0.7907 - acc: 0.2959 - val_loss: -17.8748 - val_auc: 0.5009 - val_recall: 1.0000 - val_precision: 0.8257 - val_acc: 0.2424
Epoch 2/50
33/33 [==============================] - 58s 2s/step - loss: -12.8977 - auc: 0.5004 - recall: 1.0000 - precision: 0.8224 - acc: 0.2846 - val_loss: -17.8748 - val_auc: 0.5003 - val_recall: 1.0000 - val_precision: 0.8226 - val_acc: 0.2424
Epoch 3/50
33/33 [==============================] - 58s 2s/step - loss: -12.9128 - auc: 0.5003 - recall: 1.0000 - precision: 0.8236 - acc: 0.2926 - val_loss: -17.8748 - val_auc: 0.5002 - val_recall: 1.0000 - val_precision: 0.8233 - val_acc: 0.2424
Epoch 4/50
30/33 [==========================>...] - ETA: 4s - loss: -13.0226 - auc: 0.5002 - recall: 1.0000 - precision: 0.8243 - acc: 0.2902